## Ontology
Study of how a collection of objects are related to eachother or can be grouped/subdivided, what similarity/differences exist within them, what is the hierchical relationship between them.

Information that is available does not already comes in categories that is done by individuals, thus we must decide the grouping, representations and what properties are we going to be capturing in our knowledgebase. Not everyone refers to the same object by the same name ie. different fields may have their own terminology, not all future categorization by be known, 

Ontologies may also include axioms to categorize any object/resource, but onotologies do not define any category. _Put simply in our use case ontology is the formal description of what objects exists in our information database and how we are going to represent them, their properties and grouping._

## The Semantic Web
The semantic web is a way to allow machine-interpretable knowledge to be distributed on the World Wide Web. Instead of just serving HTML pages that are meant to be read by humans, web sites will also provide information that can be used by computers.

In [1]:
import urllib.parse
import bs4 as bs
import pandas as pd

In [2]:
source = urllib.request.urlopen("http://export.arxiv.org/oai2?verb=ListRecords&set=stat&metadataPrefix=arXiv")
soup = bs.BeautifulSoup(source,"xml")

In [3]:
df = pd.DataFrame(columns=["identifier","title","abstract"])
df.head()

,identifier,title,abstract


In [4]:
records = soup.find_all("record")

In [5]:
for i in records:
    data = {"title":[''.join([k for k in i.title.text if not k.isdigit()])],
            "identifier":[i.identifier.text],
            "abstract":[i.abstract.text]}
    df = df.append(pd.DataFrame(data),sort=True)

In [6]:
df.index = range(df.shape[0])

In [7]:
df["text_corpus"]=df.title+df.abstract

In [8]:
df.head()

,abstract,identifier,title,text_corpus
0,"For the past two decades, single-index model...",oai:arXiv.org:0704.0302,Spline Single-Index Prediction Model,Spline Single-Index Prediction Model For the ...
1,"Product probability property, known in the l...",oai:arXiv.org:0704.0326,On generalized entropy measures and pathways,On generalized entropy measures and pathways ...
2,This paper deals with the investigation of t...,oai:arXiv.org:0704.0329,Solutions of fractional reaction-diffusion equ...,Solutions of fractional reaction-diffusion equ...
3,Foods naturally contain a number of contamin...,oai:arXiv.org:0704.0517,Using decomposed household food acquisitions a...,Using decomposed household food acquisitions a...
4,Three types of integral representations for ...,oai:arXiv.org:0704.0539,Integral representations for convolutions of n...,Integral representations for convolutions of n...


In [9]:
from rake_nltk import Rake
def raker(sent):
    r = Rake(max_length=2)
    r.extract_keywords_from_text(sent)
    return(r.get_ranked_phrases())

df["keywords"] = df.text_corpus.apply(raker)

In [10]:
df.head()

,abstract,identifier,title,text_corpus,keywords
0,"For the past two decades, single-index model...",oai:arXiv.org:0704.0302,Spline Single-Index Prediction Model,Spline Single-Index Prediction Model For the ...,"[yielded superior, sufficiently fast, special ..."
1,"Product probability property, known in the l...",oai:arXiv.org:0704.0326,On generalized entropy measures and pathways,On generalized entropy measures and pathways ...,"[statistical mechanics, statistical independen..."
2,This paper deals with the investigation of t...,oai:arXiv.org:0704.0329,Solutions of fractional reaction-diffusion equ...,Solutions of fractional reaction-diffusion equ...,"[specialized types, paper deals, many authors,..."
3,Foods naturally contain a number of contamin...,oai:arXiv.org:0704.0517,Using decomposed household food acquisitions a...,Using decomposed household food acquisitions a...,"[secodip panel, revisited using, pharmacokinet..."
4,Three types of integral representations for ...,oai:arXiv.org:0704.0539,Integral representations for convolutions of n...,Integral representations for convolutions of n...,"[joint distribution, integral representations,..."


In [11]:
from py2neo import Graph,Node,Relationship

In [17]:
graph = Graph()
tx = graph.begin()
for i in range(df.shape[0]):
        a = Node("PAPER",abstract=df[df.index==i]["abstract"][i],identifier=df[df.index==i]["identifier"][i],title=df[df.index==i]["title"][i])      
        tx.create(a)
        for j in range(len(df[df.index==i]["keywords"][i])):
            b = Node("Keyword",keyword=df[df.index==i]["keywords"][i][j])
            tx.create(b)
            ab = Relationship(a,"has",b)
            tx.create(ab)

In [18]:
tx.commit()

In [19]:
graph.exists(ab)

ClientError: SyntaxError: The old parameter syntax `{param}` is no longer supported. Please use `$param` instead (line 1, column 35 (offset: 34))
"OPTIONAL MATCH (a) WHERE id(a) IN {x} OPTIONAL MATCH ()-[r]->() WHERE id(r) IN {y} RETURN count(DISTINCT a) + count(DISTINCT r)"
                                   ^